### Packages

In [1]:
cd '/h/ama/workspace/ama-at-vector/best-mix'

/scratch/ssd001/home/ama/workspace/ama-at-vector/best-mix


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import math
import pickle

from mixup import mixup_graph
import time
from utils_mixup import gradmix_v2, gradmix_v2_improved

os.environ['KMP_DUPLICATE_LIB_OK']='True'
%matplotlib inline

In [4]:
from load_data import load_data_subset

labels_per_class = 5000
batch_size = 100
workers=2
dataset='cifar10'
# data_dir='/group-volume/Multimodal-Learning/ssl/vse_files/runs/fast_autoaugment/data'
data_dir='data'
valid_labels_per_class=0
mixup_alpha=0
train_loader, valid_loader, _, test_loader, num_classes = load_data_subset(batch_size,workers,dataset,data_dir,labels_per_class=labels_per_class,valid_labels_per_class=valid_labels_per_class,mixup_alpha=mixup_alpha)

Files already downloaded and verified
Files already downloaded and verified


In [5]:
''' Model '''
import models
# import torchvision.models as models
from collections import OrderedDict



model = models.__dict__['preactresnet18'](10, False, 1).cuda()

ckpt_list = []
# vanilla
checkpoint_path = '/scratch/hdd001/home/ama/mixup/2022-06-23/14723466/cifar10_arch_preactresnet18_train_vanilla_eph_300_lr_0.2_transport_eps_0.8_size_-1_job_id_7853381_seed_0'
checkpoint_path += '/checkpoint.pth.tar'
ckpt_list.append(checkpoint_path)
# #input
checkpoint_path = '/scratch/hdd001/home/ama/mixup/2022-06-23/2982229464/cifar10_arch_preactresnet18_train_mixup_eph_300_lr_0.2_m_alpha_1.0_transport_eps_0.8_size_-1_job_id_7853392_seed_1'
checkpoint_path += '/checkpoint.pth.tar'
ckpt_list.append(checkpoint_path)
# #cutmix
checkpoint_path = '/scratch/hdd001/home/ama/mixup/2022-06-23/192377623/cifar10_arch_preactresnet18_train_mixup_eph_300_lr_0.2_m_alpha_1.0_box_transport_eps_0.8_size_-1_job_id_7853387_seed_0'
checkpoint_path += '/checkpoint.pth.tar'
ckpt_list.append(checkpoint_path)
# #manifold
checkpoint_path = '/scratch/hdd001/home/ama/mixup/2022-06-23/251261079/cifar10_arch_preactresnet18_train_mixup_hidden_eph_300_lr_0.2_m_alpha_2.0_transport_eps_0.8_size_-1_job_id_7853394_seed_1'
checkpoint_path += '/checkpoint.pth.tar'
ckpt_list.append(checkpoint_path)
# #puzzle
checkpoint_path = '/scratch/hdd001/home/ama/mixup/2022-06-23/114834856/cifar10_arch_preactresnet18_train_mixup_eph_300_lr_0.2_m_alpha_1.0_graph_n_labels_3_beta_1.2_gamma_0.5_neigh_4_eta_0.2_transport_eps_0.8_size_4_job_id_7853389_seed_0'
checkpoint_path += '/checkpoint.pth.tar'
ckpt_list.append(checkpoint_path)
# #comix
checkpoint_path = '/scratch/hdd001/home/ama/mixup/2022-06-23/31093/cifar10_per_5000_arch_preactresnet18_eph_300_lr_0.12_mblock_4_mbeta_0.32_mgamma_1.0_mthres_hard0.83_meta_0.05_m_alpha_2.0_mpart_20_niter_4_omega_0.001_set_clean_1.0_seed_2'
checkpoint_path += '/checkpoint.pth.tar'
ckpt_list.append(checkpoint_path)
#ours
checkpoint_path = '/scratch/hdd001/home/ama/mixup/2022-06-30/2361913365/cifar10_arch_preactresnet18_train_ours_eph_300_lr_0.2_m_alpha_1.0_transport_eps_0.8_size_-1_job_id_7886753_seed_1'
checkpoint_path += '/checkpoint.pth.tar'
ckpt_list.append(checkpoint_path)



# mean = torch.tensor([x / 255 for x in [125.3, 123.0, 113.9]],dtype=torch.float32).reshape(1, 3, 1, 1).cuda()
# std = torch.tensor([x / 255 for x in [63.0, 62.1, 66.7]], dtype=torch.float32).reshape(1, 3, 1, 1).cuda()
# mean = torch.tensor([125.3, 123.0, 113.9])/255
# std = torch.tensor([63.0, 62.1, 66.7])/255
# mean_torch = mean.unsqueeze(0).unsqueeze(-1).unsqueeze(-1)
# std_torch = std.unsqueeze(0).unsqueeze(-1).unsqueeze(-1)



In [6]:
def test_clean(loader, model, device):
    total_loss, total_correct = 0., 0.
    for x,y in loader:
        model.eval()
        x, y = x.to(device), y.to(device)

        with torch.no_grad():
            y_hat = model(x)
            loss = torch.nn.CrossEntropyLoss()(y_hat, y)
            batch_correct = (y_hat.argmax(dim = 1) == y).sum().item()
        
        total_correct += batch_correct
        total_loss += loss.item() * x.shape[0]
        
    test_acc = total_correct / len(loader.dataset) * 100
    test_loss = total_loss / len(loader.dataset)
    return test_acc, test_loss

def test_clean(loader, model, device):
    total_loss, total_correct = 0., 0.
    for x,y in loader:
        model.eval()
        x, y = x.to(device), y.to(device)

        with torch.no_grad():
            y_hat = model(x)
            loss = torch.nn.CrossEntropyLoss()(y_hat, y)
            batch_correct = (y_hat.argmax(dim = 1) == y).sum().item()
        
        total_correct += batch_correct
        total_loss += loss.item() * x.shape[0]
        
    test_acc = total_correct / len(loader.dataset) * 100
    test_loss = total_loss / len(loader.dataset)
    return test_acc, test_loss

def test_fgsm_linf(loader, model, device):
    total_loss, total_correct = 0., 0.
    for x,y in loader:
        model.eval()
        x, y = x.to(device), y.to(device)
        delta = torch.zeros_like(x, requires_grad = True).to(device)

#         with torch.no_grad():
        y_hat = model(x+delta)
        loss = torch.nn.CrossEntropyLoss()(y_hat, y)
        
        loss.backward()
        
        delta.data = 8./255 *(62/255) * delta.grad.sign()
        y_hat = model(x+delta.detach())
        loss = torch.nn.CrossEntropyLoss()(y_hat, y)
        
        batch_correct = (y_hat.argmax(dim = 1) == y).sum().item()
        
        total_correct += batch_correct
        total_loss += loss.item() * x.shape[0]
        
    test_acc = total_correct / len(loader.dataset) * 100
    test_loss = total_loss / len(loader.dataset)
    return test_acc, test_loss

def test_fgsm_l2(loader, model, device):
    total_loss, total_correct = 0., 0.
    for x,y in loader:
        model.eval()
        x, y = x.to(device), y.to(device)
        delta = torch.zeros_like(x, requires_grad = True).to(device)

#         with torch.no_grad():
        y_hat = model(x+delta)
        loss = torch.nn.CrossEntropyLoss()(y_hat, y)
        
        loss.backward()
        
        delta.data = 0.5 *(62/255) * delta.grad/torch.norm(delta.grad.detach(),p=2,dim=[1,2,3],keepdim=True)
        y_hat = model(x+delta.detach())
        loss = torch.nn.CrossEntropyLoss()(y_hat, y)
        
        batch_correct = (y_hat.argmax(dim = 1) == y).sum().item()
        
        total_correct += batch_correct
        total_loss += loss.item() * x.shape[0]
        
    test_acc = total_correct / len(loader.dataset) * 100
    test_loss = total_loss / len(loader.dataset)
    return test_acc, test_loss


def test_gaussian(loader, model, var, device):
    total_loss, total_correct = 0., 0.
    for x,y in loader:
        model.eval()
        x, y = x.to(device), y.to(device)
        
        noise = (var**0.5)*torch.randn_like(x, device = x.device)
#         ipdb.set_trace()

        with torch.no_grad():
            y_hat = model(x+noise)
            loss = torch.nn.CrossEntropyLoss()(y_hat, y)
            batch_correct = (y_hat.argmax(dim = 1) == y).sum().item()
        
        total_correct += batch_correct
        total_loss += loss.item() * x.shape[0]
        
    test_acc = total_correct / len(loader.dataset) * 100
    test_loss = total_loss / len(loader.dataset)
    return test_acc, test_loss

CORRUPTIONS_CIFAR10=['brightness',         
'gaussian_noise',    
'saturate',
'contrast',           
'glass_blur',        
'shot_noise',
'defocus_blur',       
'impulse_noise',     
'snow',
'elastic_transform',  
'jpeg_compression',  
'spatter',
'fog',         
'speckle_noise',
'frost',              
'motion_blur',       
'zoom_blur',
'gaussian_blur',      
'pixelate'] 

def eval_corrupt(model, severity, device):
#     dct_matrix = getDCTmatrix(28)
    acc = []
    total_correct = 0
    
    corruptions_list = CORRUPTIONS_CIFAR10
    
    for corrupt_type in corruptions_list:
        data_path = '/h/ama/workspace/ama-at-vector/freq-robust/data/CIFAR-10-C/' + corrupt_type + '.npy'
        label_path = '/h/ama/workspace/ama-at-vector/freq-robust/data/CIFAR-10-C/labels.npy'
        x = torch.tensor(np.transpose(np.load(data_path), (0, 3, 1, 2))/255.,dtype= torch.float32)[(severity-1)*10000:severity*10000].to(device)
        y = torch.tensor(np.load(label_path),dtype=torch.float32)[(severity-1)*10000:severity*10000].to(device)

        for i in range(10):
            with torch.no_grad():
                y_hat = model(x[1000*i:1000*(i+1)])
                total_correct += (y_hat.argmax(dim = 1) == y[1000*i:1000*(i+1)]).sum().item()
        
        corrupt_acc = total_correct / len(y) * 100
        acc.append(corrupt_acc)
        total_correct = 0
        
        del x,y 
    
    return acc

In [7]:
for ckpt in ckpt_list:
    checkpoint = torch.load(ckpt)


    od = OrderedDict()
    for key in checkpoint['state_dict'].keys():
        od[key[7:]] = checkpoint['state_dict'][key]
    model.load_state_dict(od)
    
#     print(test_clean(test_loader, model, 'cuda:0'))
#     print(test_fgsm_linf(test_loader, model, 'cuda:0'))
#     print(test_fgsm_l2(test_loader, model, 'cuda:0'))
#     print(test_gaussian(test_loader, model, 0.1, 'cuda:0'))
#     print(test_gaussian(test_loader, model, 0.1, 'cuda:0'))
#     print(test_gaussian(test_loader, model, 0.5, 'cuda:0'))
    print(np.array(eval_corrupt(model, 3, 'cuda:0')).mean())

78.58263157894737
85.92526315789476
77.15368421052632
83.42263157894736
79.19842105263159
79.19526315789474
80.56736842105263
